In [ ]:
from math import hypot
cap=cv2.VideoCapture(0)
detector=dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
sunglass=cv2.imread("tempsnip.png")

In [1]:
while True:
    _,frame=cap.read()
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces=detector(gray_frame)
    
    for face in faces:
        landmarks = predictor(gray_frame, face)
        extreme_left=(landmarks.part(17).x,landmarks.part(17).y)                   
        extreme_right=(landmarks.part(26).x,landmarks.part(26).y)              
        center_eye=(landmarks.part(27).x,landmarks.part(27).y)
        left_eye=(landmarks.part(36).x,landmarks.part(36).y)
        right_eye=(landmarks.part(45).x,landmarks.part(45).y)
        bottom_eye=(landmarks.part(29).x,landmarks.part(29).y)

        new_height=int(hypot(bottom_eye[0]-center_eye[0],bottom_eye[1]-center_eye[1]))
        frame_width=int(hypot(extreme_left[0]-extreme_right[0],extreme_left[1]-extreme_right[1]))
       
        eye_width=int(hypot(extreme_left[0]-extreme_right[0],extreme_left[1]-extreme_right[1]))

        eye_height=int(new_height*2)               

        top_left = (int(extreme_left[0]),int(extreme_left[1]))
        bottom_right = (int(extreme_right[0]),int(extreme_right[1] + (new_height*2)))
        
        
        
        size_sunglass=cv2.resize(sunglass,(eye_width,eye_height))  #reshape sunglass to person eyes
        sunglass_gray= cv2.cvtColor(size_sunglass, cv2.COLOR_BGR2GRAY) #converting it to gray
        _,sunglass_mask = cv2.threshold(sunglass_gray, 40, 255, cv2.THRESH_BINARY_INV)


        eye_area = frame[top_left[1]: top_left[1] + int(new_height*2),top_left[0]: top_left[0] + frame_width]
        no_eye_area = cv2.bitwise_and(eye_area, eye_area,mask=sunglass_mask)   
        final_eye = cv2.add(no_eye_area,size_sunglass)  

        frame[top_left[1]: top_left[1] + (new_height*2),top_left[0]: top_left[0] + frame_width]=final_eye


    cv2.imshow("Frame", frame)



    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()